In [159]:
import re
import json

def extract_json_using_patterns(text):
    """使用一组正则表达式模式来提取 JSON"""
    # 保留原始文本的换行符
    text = text.strip()
    print("text:", text)
    
    patterns = [
        r'\{[\s\S]*\}',  # 新的正则表达式模式，匹配第一个 '{' 和最后一个 '}' 之间的内容
        # 保留其他原有模式
        r'(\{[\s\S]*?\})\s*\}$',
        r'\{\s*"result"\s*:\s*\[[\s\S]*?\]\s*\}',
        r'"""json\s*(\{[\s\S]*?\})\s*"""',
        r'```json\s*(\{[\s\S]*?\})\s*```',
    ]

    # 依次尝试使用每个正则表达式模式进行匹配
    for pattern in patterns:
        match = re.search(pattern, text, re.DOTALL)
        if match:
            # 检查正则表达式是否使用了捕获组
            if match.lastindex:
                json_str = match.group(1)  # 获取捕获组中的内容
            else:
                json_str = match.group(0)  # 获取整个匹配的内容
            print(f"匹配到的 JSON: {json_str}")
            try:
                # 尝试将提取到的字符串解析为 JSON
                result_data = json.loads(json_str)
                return result_data
            except json.JSONDecodeError as e:
                # 如果 JSON 解析失败，继续尝试下一个模式
                print(f"JSON 解析失败: {e}")
                continue

    # 如果没有找到匹配的 JSON 数据
    print("未找到符合模式的 JSON 数据")
    return {}

# 测试代码
text = '''
一些前置文本
{
    "result": [
        {"type": "发起", "content": "具你，为什么要我买？这是第一套。"},
        {"type": "其它", "content": ""}
    ],
    "compliance": "低"
}
一些后置文本
'''

result = extract_json_using_patterns(text)
print("提取结果:", result)


text: 一些前置文本
{
    "result": [
        {"type": "发起", "content": "具你，为什么要我买？这是第一套。"},
        {"type": "其它", "content": ""}
    ],
    "compliance": "低"
}
一些后置文本
匹配到的 JSON: {
    "result": [
        {"type": "发起", "content": "具你，为什么要我买？这是第一套。"},
        {"type": "其它", "content": ""}
    ],
    "compliance": "低"
}
提取结果: {'result': [{'type': '发起', 'content': '具你，为什么要我买？这是第一套。'}, {'type': '其它', 'content': ''}], 'compliance': '低'}


In [157]:
extract_json_using_patterns(text)

text: ```json
{
"result":
[{"type":"发起","content": "具你，为什么要我买？这是第一套。"},
{"type":"其它","content": ""}],
"compliance": "低"
}
匹配到的 JSON: {
"result":
[{"type":"发起","content": "具你，为什么要我买？这是第一套。"}
JSON 解析失败: Expecting ',' delimiter: line 3 column 45 (char 56)
未找到符合模式的 JSON 数据


{}

In [158]:
{
"result":
[{"type":"发起","content": "具你，为什么要我买？这是第一套。"},
{"type":"其它","content": ""}],
"compliance": "低"
}

{'result': [{'type': '发起', 'content': '具你，为什么要我买？这是第一套。'},
  {'type': '其它', 'content': ''}],
 'compliance': '低'}

In [176]:
import pandas as pd 
def parse_test_result(test_result):
    """解析模型返回的结果，提取 JSON"""
    if test_result:
        try:
            result_data = json.loads(test_result)
            
            return result_data
        except json.JSONDecodeError:
            return extract_json_using_patterns(test_result)
    else:
        return {}
def remove_punctuation(text):
    # 中英文标点符号的正则表达式
    punctuation_pattern = r'[，。！？、；：“”‘’（）《》【】…—]+' + r'[.,\/#!$%\^&\*;:{}=\-_`~()]'
    # 使用正则表达式替换标点符号为空字符串
    no_punctuation_text = re.sub(punctuation_pattern, '', text)
    return no_punctuation_text

output_result = [[{"start_time": 27300, "end_time": 32940, "text": "具你，为什么要我买？这是第一套。", "label": 0}, {"start_time": 35310, "end_time": 39510, "text": "喂，你，吃你吃你狗，你，", "label": 1}, {"model_input": "\n                “老师话语”：具你，为什么要我买？这是第一套。 \n                “学生话语”：喂，你，吃你吃你狗，你，\n                "}, {"soikit_test_result": "{\n  \"result\": [\n    {\"type\": \"发起\", \"content\": \"具你，为什么要我买？这是第一套。\"}\n  ],\n  \"compliance\": \"低\"\n}"}], [{"start_time": 40560, "end_time": 42710, "text": "好，把语文书翻到第50页，", "label": 0}, {"start_time": 40560, "end_time": 42710, "text": "", "label": 1}, {"model_input": "“老师话语”：好，把语文书翻到第50页， "}, {"soikit_test_result": "{\n  \"result\": [\n    {\"type\": \"其它\", \"content\": \"好，把语文书翻到第50页，\"}\n  ],\n  \"compliance\": \"无\"\n}"}], [{"start_time": 45590, "end_time": 47190, "text": "然后铅笔收起来把，", "label": 0}, {"start_time": 47910, "end_time": 49590, "text": "课堂练习放到左上角，", "label": 1}, {"model_input": "\n                “老师话语”：然后铅笔收起来把， \n                “学生话语”：课堂练习放到左上角，\n                "}, {"soikit_test_result": "{\n  \"result\": [\n    {\"type\": \"其它\", \"content\": \"然后铅笔收起来吧\"}\n  ],\n  \"compliance\": \"无\"\n}"}], [{"start_time": 50070, "end_time": 52760, "text": "先把语文书翻到翻到第50页，翻到这里，", "label": 0}, {"start_time": 52780, "end_time": 52790, "text": "没有，50。我现在这个阳猫世，", "label": 1}, {"model_input": "\n                “老师话语”：先把语文书翻到翻到第50页，翻到这里， \n                “学生话语”：没有，50。我现在这个阳猫世，\n                "}, {"soikit_test_result": "{\n  \"result\": [\n    {\"type\": \"其它\", \"content\": \"先把语文书翻到翻到第50页，翻到这里，\"}\n  ],\n  \"compliance\": \"无\"\n}"}], [{"start_time": 53000, "end_time": 69880, "text": "我看谁今天坐姿有问题啊啊，", "label": 0}, {"start_time": 53000, "end_time": 69880, "text": "", "label": 1}, {"model_input": "“老师话语”：我看谁今天坐姿有问题啊啊， "}, {"soikit_test_result": "{\n\"result\": [\n  {\"type\":\"发起\",\"content\":\"我看谁今天坐姿有问题啊啊，\"}\n],\n\"compliance\": \"无\"\n}"}]]
model_name = "soikit_test"
# def process_output_result(output_result, model_name):
"""处理模型的输出结果，生成最终的 DataFrame"""
result_list = []

for sublist in output_result:
    print("sub_list",sublist)
    test_result = None
    if len(sublist) >= 4:
        third_item = sublist[3]
        test_result = next(iter(third_item.values()))
    else:
        test_result = None
    
    contents = parse_test_result(test_result)
    print("contents",contents)
    new_sublist = []
    if not contents:
        for item in sublist:
            print("item",item)
            if 'start_time' in item:
                new_item = item.copy()
                new_item[f'{model_name}_predict'] = '{}'
                new_sublist.append(new_item)
        result_list.append(new_sublist)
        continue
    for item in sublist:
        if 'start_time' in item:
            new_item = item.copy()

            match_found = False
            for content in contents["result"]:
                if content['content'] != "":
                    if remove_punctuation(content['content']) in remove_punctuation(item['text']):
                        new_item[f'{model_name}_predict'] = test_result
                        match_found = True
                        break
            if not match_found:
                new_item[f'{model_name}_predict'] = ''

            new_sublist.append(new_item)
    result_list.append(new_sublist)
    # return df_result

sub_list [{'start_time': 27300, 'end_time': 32940, 'text': '具你，为什么要我买？这是第一套。', 'label': 0}, {'start_time': 35310, 'end_time': 39510, 'text': '喂，你，吃你吃你狗，你，', 'label': 1}, {'model_input': '\n                “老师话语”：具你，为什么要我买？这是第一套。 \n                “学生话语”：喂，你，吃你吃你狗，你，\n                '}, {'soikit_test_result': '{\n  "result": [\n    {"type": "发起", "content": "具你，为什么要我买？这是第一套。"}\n  ],\n  "compliance": "低"\n}'}]
contents {'result': [{'type': '发起', 'content': '具你，为什么要我买？这是第一套。'}], 'compliance': '低'}
sub_list [{'start_time': 40560, 'end_time': 42710, 'text': '好，把语文书翻到第50页，', 'label': 0}, {'start_time': 40560, 'end_time': 42710, 'text': '', 'label': 1}, {'model_input': '“老师话语”：好，把语文书翻到第50页， '}, {'soikit_test_result': '{\n  "result": [\n    {"type": "其它", "content": "好，把语文书翻到第50页，"}\n  ],\n  "compliance": "无"\n}'}]
contents {'result': [{'type': '其它', 'content': '好，把语文书翻到第50页，'}], 'compliance': '无'}
sub_list [{'start_time': 45590, 'end_time': 47190, 'text': '然后铅笔收起来把，', 'label': 0}, {'start_ti

In [174]:
sub_list  = [{'start_time': 27300, 'end_time': 32940, 'text': '具你，为什么要我买？这是第一套。', 'label': 0}, {'start_time': 35310, 'end_time': 39510, 'text': '喂，你，吃你吃你狗，你，', 'label': 1}, {'model_input': '\n                “老师话语”：具你，为什么要我买？这是第一套。 \n                “学生话语”：喂，你，吃你吃你狗，你，\n                '}, {'soikit_test_result': '{\n  "result": [\n    {"type": "发起", "content": "具你，为什么要我买？这是第一套。"}\n  ],\n  "compliance": "低"\n}'}]
sub_list[0]

{'start_time': 27300,
 'end_time': 32940,
 'text': '具你，为什么要我买？这是第一套。',
 'label': 0}

In [ ]:
if "start_time" in sub_list[0]:
    print(1)

In [ ]:
    


    for item in sublist:
        if 'start_time' in item:
            new_item = item.copy()

            match_found = False
            for content in contents["result"]:
                if content['content'] != "":
                    if content['content'] in item['text']:
                        new_item[f'{model_name}_predict'] = test_result
                        match_found = True
                        break
            if not match_found:
                new_item[f'{model_name}_predict'] = ''

            new_sublist.append(new_item)
    result_list.append(new_sublist)

flat_list = [item for sublist in result_list for item in sublist]
df_result = pd.DataFrame(flat_list)

In [189]:
"\n".join(['发起：老师发起提问1老师讲解内容1老师发起提问2\n回应：学生回答1', '讲解：老师讲解内容2\n讲解：老师讲解内容3\n发起：老师发起提问3老师发起提问4\n回应：学生回答2\n讲解：老师讲解内容4\n讲解：老师讲解内容5'])


'发起：老师发起提问1老师讲解内容1老师发起提问2\n回应：学生回答1\n讲解：老师讲解内容2\n讲解：老师讲解内容3\n发起：老师发起提问3老师发起提问4\n回应：学生回答2\n讲解：老师讲解内容4\n讲解：老师讲解内容5'

In [190]:
# 根据讲解分割点来构造主题提取prompt的输入文本

In [215]:
input1 = '发起：老师发起提问1老师讲解内容1老师发起提问2\n回应：学生回答1\n讲解：老师讲解内容2\n讲解：老师讲解内容3\n发起：老师发起提问3老师发起提问4\n回应：学生回答2\n讲解：老师讲解内容4\n讲解：老师讲解内容5\n发起：老师发起提问1老师讲解内容1老师发起提问2\n回应：学生回答1\n讲解：老师讲解内容2\n讲解：老师讲解内容3\n发起：老师发起提问3老师发起提问4\n回应：学生回答2\n讲解：老师讲解内容4\n讲解：老师讲解内容5'

In [216]:
input1_list = input1.split("\n")
input1_list

['发起：老师发起提问1老师讲解内容1老师发起提问2',
 '回应：学生回答1',
 '讲解：老师讲解内容2',
 '讲解：老师讲解内容3',
 '发起：老师发起提问3老师发起提问4',
 '回应：学生回答2',
 '讲解：老师讲解内容4',
 '讲解：老师讲解内容5',
 '发起：老师发起提问1老师讲解内容1老师发起提问2',
 '回应：学生回答1',
 '讲解：老师讲解内容2',
 '讲解：老师讲解内容3',
 '发起：老师发起提问3老师发起提问4',
 '回应：学生回答2',
 '讲解：老师讲解内容4',
 '讲解：老师讲解内容5']

In [217]:
splitpoint = {
  "breakpoint": [
    "讲解:老师讲解内容2",
    "讲解:老师讲解内容3",
    '讲解：老师讲解内容3',
  ]
}

In [218]:
splitpoint.get("breakpoint")

['讲解:老师讲解内容2', '讲解:老师讲解内容3', '讲解：老师讲解内容3']

In [219]:
#找到['讲解:老师讲解内容2', '讲解:老师讲解内容3']中值在输入中的索引，并对这个输入进行切割 

In [220]:
import string

def remove_punctuation(text):
    # 定义中英文标点符号
    punctuation = string.punctuation + '，。；：“”‘’？！《》、'
    # 使用正则表达式去除标点
    text = re.sub(f"[{re.escape(punctuation)}]", '', text)
    return text


In [221]:
list1 = []

for i, item in enumerate(input1_list):
    for item2 in splitpoint.get("breakpoint"):
        
        if remove_punctuation(item) == remove_punctuation(item2):
            list1.append(i)

In [231]:
list1

[2, 3, 3, 10, 11, 11]

In [232]:
# 列表去重
def method_dict(lst):
    unique_lst = list(dict.fromkeys(lst))
    return unique_lst
method_dict(list1)

[2, 3, 10, 11]

[10, 11, 2, 3]

In [208]:
[0,2,3,10,11]

In [223]:
[0,2,10]

[0, 2, 10]

In [226]:
first_piece = input1_list[0:2]
first_piece

['发起：老师发起提问1老师讲解内容1老师发起提问2', '回应：学生回答1']

In [228]:
second_piece = input1_list[2:10]
second_piece

['讲解：老师讲解内容2',
 '讲解：老师讲解内容3',
 '发起：老师发起提问3老师发起提问4',
 '回应：学生回答2',
 '讲解：老师讲解内容4',
 '讲解：老师讲解内容5',
 '发起：老师发起提问1老师讲解内容1老师发起提问2',
 '回应：学生回答1']

In [229]:
third_piece = input1_list[10:]
third_piece

['讲解：老师讲解内容2',
 '讲解：老师讲解内容3',
 '发起：老师发起提问3老师发起提问4',
 '回应：学生回答2',
 '讲解：老师讲解内容4',
 '讲解：老师讲解内容5']

In [209]:
remove_punctuation("讲解:老师讲解内容3")

'讲解老师讲解内容3'

In [ ]:
# 如果没有分割点，则要将原始文本按照“发起”作为切割 点，将原始文本分割多个师生对话序列。

In [250]:
import re

# 定义函数来去除中英文标点符号
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

# 1. 将prompt2的输入文本按照\n进行分割
prompt2_input = """发起：老师发起提问1老师讲解内容1老师发起提问2\n回应：学生回答1\n讲解：老师讲解内容2\n讲解：老师讲解内容3\n发起：老师发起提问3老师发起提问4\n回应：学生回答2\n讲解：老师讲解内容4\n讲解：老师讲解内容5"""
input1_list = prompt2_input.split('\n')

input1_list

['发起：老师发起提问1老师讲解内容1老师发起提问2',
 '回应：学生回答1',
 '讲解：老师讲解内容2',
 '讲解：老师讲解内容3',
 '发起：老师发起提问3老师发起提问4',
 '回应：学生回答2',
 '讲解：老师讲解内容4',
 '讲解：老师讲解内容5']

In [268]:
# 2. 定义breakpoint列表，寻找索引
breakpoint = ["讲解：老师讲解内容2", "讲解：老师讲解内容3"]
list1 = []

for i, item in enumerate(input1_list):
    for item2 in breakpoint:
        # 去除中英文标点符号后再匹配
        if remove_punctuation(item.strip()) == remove_punctuation(item2.strip()):
            list1.append(i)
list1

[2, 3]

In [270]:
def filter_consecutive(lst):
    if not lst:
        return []

    # 初始化结果列表
    result = [lst[0]]
    
    # 遍历列表，检查是否连续
    for i in range(1, len(lst)):
        if lst[i] != lst[i - 1] + 1:
            result.append(lst[i])

    return result

# 测试代码
print(filter_consecutive([2]))          # 输出: [2, 10]
print(filter_consecutive([2,10,11]))            # 输出: [2, 10]
print(filter_consecutive([2,3,10]))             # 输出: [2, 10]
print(filter_consecutive([2,3,10,11,15]))       # 输出: [2, 10, 15]
print(filter_consecutive([2,3,5,10,11,12,13,14,15,17])) # 输出: [2, 10, 15]



[2]
[2, 10]
[2, 10]
[2, 10, 15]
[2, 5, 10, 17]


In [271]:
# 3. 对索引列表list1去重
def method_dict(lst):
    return list(dict.fromkeys(lst))

list1 = method_dict(list1)

# 4. 处理相邻索引，保留较小的索引生成新的
def filter_consecutive(lst):
    if not lst:
        return []

    # 初始化结果列表
    result = [lst[0]]
    
    # 遍历列表，检查是否连续
    for i in range(1, len(lst)):
        if lst[i] != lst[i - 1] + 1:
            result.append(lst[i])

    return result
# 索引列表list2
list2 = filter_consecutive(list1)
list2

[2]

In [277]:


list2 = [2]

# 5. 根据list2中的索引切分input1_list
pieces = []
if list2:
    # 第一段从开始到第一个list2的索引
    pieces.append(input1_list[0:list2[0]])
    # 中间段落
    for i in range(len(list2) - 1):
        pieces.append(input1_list[list2[i]:list2[i+1]])
    # 最后一段
    pieces.append(input1_list[list2[-1]:])
else:
    # 如果list2为空，则不进行切分，直接使用整个input1_list
    pieces.append(input1_list)
pieces

[['发起：老师发起提问1老师讲解内容1老师发起提问2', '回应：学生回答1'],
 ['讲解：老师讲解内容2',
  '讲解：老师讲解内容3',
  '发起：老师发起提问3老师发起提问4',
  '回应：学生回答2',
  '讲解：老师讲解内容4',
  '讲解：老师讲解内容5']]

In [278]:

# 6. 合并子列表中的相邻讲解内容
def merge_explanations(piece):
    new_piece = []
    merged = ""
    for item in piece:
        if item.startswith("讲解："):
            merged += item.replace("讲解：", "") + ","
        else:
            if merged:
                new_piece.append("讲解：" + merged[:-1])  # 去掉最后一个逗号
                merged = ""
            new_piece.append(item)
    if merged:
        new_piece.append("讲解：" + merged[:-1])
    return new_piece

# 对每个piece进行相邻讲解内容的合并
merged_pieces = [merge_explanations(piece) for piece in pieces]
merged_pieces

[['发起：老师发起提问1老师讲解内容1老师发起提问2', '回应：学生回答1'],
 ['讲解：老师讲解内容2,老师讲解内容3', '发起：老师发起提问3老师发起提问4', '回应：学生回答2', '讲解：老师讲解内容4,老师讲解内容5']]

In [285]:

# 7. 分割合并后的子列表
list_all = []

def split_by_initiate(list_all):
    segments = []
    current_segment = []

    for item in list_all:
        if item.startswith('发起：'):
            if current_segment:  # 如果当前段落不为空，则保存之前的段落
                segments.append(current_segment)
                current_segment = []  # 重置当前段落
        current_segment.append(item)  # 将当前项添加到当前段落

    # 处理最后一个段落
    if current_segment:
        segments.append(current_segment)
    #  segments 中第一个子列表的开头是否是“讲解：”，如果是，则将第二个子列表的内容添加到第一个子列表后面。
    if segments[0][0].startswith('讲解：'):
    # 将第二个子列表的内容扩展到第一个子列表后面
        segments[0].extend(segments[1])
        segments.pop(1)
    return segments

# 检查breakpoint是否为空
if not breakpoint:
    # 如果为空，直接分割input1_list
    list_all.extend(split_by_initiate(input1_list))
else:
    for merged_piece in merged_pieces:
        list_all.extend(split_by_initiate(merged_piece))
list_all


[['发起：老师发起提问1老师讲解内容1老师发起提问2', '回应：学生回答1'],
 ['讲解：老师讲解内容2,老师讲解内容3', '发起：老师发起提问3老师发起提问4', '回应：学生回答2', '讲解：老师讲解内容4,老师讲解内容5']]

In [288]:
# 8. 拼成prompt3的输入文本
prompt3_input = ""
for idx, segment in enumerate(list_all, start=1):
    prompt3_input += f"师生对话{idx}\n" + "\n".join(segment) + "\n\n"

# 输出结果
print(prompt3_input.strip())

<class 'str'>


In [284]:
def split_by_initiate(list_all):
    segments = []
    current_segment = []

    for item in list_all:
        if item.startswith('发起：'):
            if current_segment:  # 如果当前段落不为空，则保存之前的段落
                segments.append(current_segment)
                current_segment = []  # 重置当前段落
        current_segment.append(item)  # 将当前项添加到当前段落

    # 处理最后一个段落
    if current_segment:
        segments.append(current_segment)
    #  segments 中第一个子列表的开头是否是“讲解：”，如果是，则将第二个子列表的内容添加到第一个子列表后面。
    if segments[0][0].startswith('讲解：'):
    # 将第二个子列表的内容扩展到第一个子列表后面
        segments[0].extend(segments[1])
        segments.pop(1)
    return segments

# 测试输入
list_all = [
    
    '发起：老师发起提问3老师发起提问4',
    '回应：学生回答2',
    '发起：老师发起提问1老师讲解内容1老师发起提问2',
    '回应：学生回答1'
]

# 调用函数并打印结果
output = split_by_initiate(list_all)
output 
# for i, segment in enumerate(output, 1):
#     print(f"段落 {i}:")
#     print("\n".join(segment))
#     print()  # 添加空行分隔段落



[['发起：老师发起提问3老师发起提问4', '回应：学生回答2'], ['发起：老师发起提问1老师讲解内容1老师发起提问2', '回应：学生回答1']]

In [282]:
list2 = output 

# 检查第一个子列表的文本开头
if list2[0][0].startswith('讲解：'):
    # 将第二个子列表的内容扩展到第一个子列表后面
    list2[0].extend(list2[1])
    list2.pop(1)

# 打印结果
print(list2)


[['讲解：在dfdf 发起：老师发起提问3老师发起提问4', '回应：学生回答2', '发起：老师发起提问1老师讲解内容1老师发起提问2', '回应：学生回答1']]
